In [ ]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)
import matplotlib.pyplot as plt
import seaborn as sns
import datetime as dt
import dateutil.relativedelta
#from datetime import date
from geopy.geocoders import Nominatim
from pandas_profiling import ProfileReport
from geopy.extra.rate_limiter import RateLimiter
#!conda install -c conda-forge folium=0.5.0 --yes
import folium
from folium import Choropleth, Circle, Marker
from folium.plugins import HeatMap, MarkerCluster
# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

# You can write up to 5GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

print("Setup Complete")

In [ ]:
#Télécharger les données
my_filepath='../input/sales-forecasting/train.csv'
donne = pd.read_csv(my_filepath,index_col='Order ID')
donne

In [ ]:
donne1=donne


In [ ]:
#Mettre les date (string) en date

# Convertion de str en dates
donne1['Order Date'] =  pd.to_datetime(donne1['Order Date'])
donne1['Ship Date'] =  pd.to_datetime(donne1['Ship Date'])

# Intervertir les jours et les mois dans notre cellule de date 
donne1['Order Date'] = donne1['Order Date'].apply(lambda x: dt.datetime.strftime(x, '%d-%m-%Y'))
donne1['Ship Date'] = donne1['Ship Date'].apply(lambda x: dt.datetime.strftime(x, '%d-%m-%Y'))

# Re-convertion de str en dates
donne1['Order Date'] =  pd.to_datetime(donne1['Order Date'])
donne1['Ship Date'] =  pd.to_datetime(donne1['Ship Date'])
donne1['Order Date']

In [ ]:
#modification de my_data

donne1['Délai entre commande et envoie']= donne1['Ship Date']-donne1['Order Date'] # marche pas
donne1['Délai entre commande et envoie'] = donne1['Délai entre commande et envoie'].dt.days.astype('int16')

donne1.loc[(donne1['Délai entre commande et envoie'])<0]
#on a 0 index a enlever car les dates ont ete bien rentre

In [ ]:
donne1['Ship Mode'].unique()
donne1['Customer ID'].nunique()#on a 793 clients differents
donne1['Customer Name'].nunique() #793, meme chose yeah on peut l'enlever
donne1['Segment'].unique()
donne1['Country'].unique()#juste usa on peut la supprimer
donne1.drop(columns=['Country'])
donne1['City'].nunique() #clients sont dans 529 villes
donne1['State'].nunique()#dans 49 etats
donne1['Product ID'].nunique()#on vend 1861 produits
donne1['Product Name'].nunique()#  mais 1849 noms differents
donne1['Category'].unique()# 3 categories
donne1['Sub-Category'].nunique()#17 sous categories
donne1['mois']=donne1['Order Date'].dt.month
donne1['jour']=donne1['Order Date'].dt.day
donne1['Sales'].describe()

In [ ]:
#code postaux
donne1['Postal Code'].isna().any()
#donc il y a des na dans Postal Code
donne1 = donne1[donne1['Postal Code'].notna()]
donne1['Postal Code'].isna().any()
donne1['Postal Code']=donne1['Postal Code'].astype(int).astype(str)
#il n'y a pus de na on a donc perdu 11 rows et on a transformer les codes postaux en int (enlever le 0) ensuite en 
#strpour avoir des données qui s'analyse mieux. car les codes postaux sont plus utile en données catégoriques que numérique
donne1

In [ ]:
#délai de traitement par code postaux, ca vaudrait la peine de regarder par item aussi
donne1['Délai entre commande et envoie'].max()# valeur max est 7, on veut savoir combien il y en a et les selectionner
top=donne1.loc[(donne1['Délai entre commande et envoie']>=7)]# 615 on le delai le plus long de 7 jours
donne1['Délai entre commande et envoie'].min()
bottom=donne1.loc[(donne1['Délai entre commande et envoie']==0)] # 514 shipper le jour meme 
#donne1['Délai entre commande et envoie']=donne1['Délai entre commande et envoie'].astype(int)
dt_state_delai= donne1
dt_state_delai=dt_state_delai.set_index('State')
dt_state_delai=dt_state_delai['Délai entre commande et envoie']
#dt_state_delai=dt_state_delai.groupby('State')
dt_state_delai.groupby(['State']).agg([len, min, max,'mean','std']).sort_values(by='len',ascending=False)

#dt_state_delai[15:30]
#sns.barplot(x=dt_state_delai.index, y=dt_state_delai[:])
#bottom10=donne1.nlargest(10,'Délai entre commande et envoie')

In [ ]:
# tableau de bord
#profile = ProfileReport(donne1,
#                        title='Pandas Profiling Report',
#                        html={'style':{'full_width':True}})
#profile.to_widgets()
#profile.to_notebook_itframe()

In [ ]:
# adresse complete les 793 clients sont repartis dans 627 codes postaux
adresse=donne1
adresse['adresse complete']=donne1['State']+','+donne1['City']+','+donne1['Postal Code']
adresse=pd.DataFrame(data=adresse).rename(columns={0:'adresse complete'}).reset_index().dropna().drop('Order ID',axis=1)
#adresse.drop_duplicates(subset='adresse complete')
adresse=adresse.groupby('adresse complete').Sales.sum()
adresse=pd.DataFrame(data=adresse).reset_index()
adresse

In [ ]:
locator = Nominatim(user_agent='kevhervieux@gmail.com')
geocode = RateLimiter(locator.geocode, min_delay_seconds=1)#on pourrait etre beaucoup plus rapide mais on doit payer donc delai min = 1 sec
adresse['location'] = adresse['adresse complete'].apply(geocode)
adresse['point'] = adresse['location'].apply(lambda loc: tuple(loc.point) if loc else None)
adresse[['latitude', 'longitude', 'altitude']] = pd.DataFrame(adresse['point'].tolist(), index=adresse.index)
adresse_backup=adresse

In [ ]:
adresse=adresse_backup
adresse=adresse[adresse['longitude'].notnull()]
adresse=adresse[adresse['latitude'].notnull()]
adresse=adresse.reset_index()
adresse

In [ ]:
#map marker
us_map=folium.Map(location=[37.0902, -95.7129], zoom_start=4,tiles='cartodbpositron')
mc=MarkerCluster()
client=folium.map.FeatureGroup()
for lat, lng, in zip(adresse['latitude'], adresse['longitude']):
    client.add_child(
        folium.features.Marker([lat, lng]))
# add incidents to map
us_map.add_child(client)
us_map

In [ ]:
#heat map
us_heat_map=folium.Map(location=[37.0902, -95.7129], zoom_start=4, tiles='cartodbpositron')
HeatMap(data=adresse[['latitude','longitude']],radius=15).add_to(us_heat_map)#j<ai essaye 5-10-15-20-25 et 15 c<est le plus sexy
us_heat_map

In [ ]:
#cluster map
#for lat,lng in zip(adresse['latitude'], adresse['longitude']):
    #if not math.isnan(adresse['longitude']) and not math.isnan(adresse['latitude']):
        #mc.add_child(Marker([adresse['latitude'],adresse['longitude']]))


In [ ]:
#suivi des clients
date_today=donne1['Order Date'].max()

def client_perdu(data_frame,nom_colonne,mois_derniere_commande,annee_derniere_commande,colonne_client):#t_derniere_commande est en mois
    f = dateutil.relativedelta.relativedelta(months=mois_derniere_commande)
    g = dateutil.relativedelta.relativedelta(years=annee_derniere_commande)
    date_max=date_today-f-g
    date_max=np.datetime64(date_max)
    x= data_frame.loc[data_frame[nom_colonne]>=date_max]
    x=x[colonne_client]
    x=pd.DataFrame(data=x).drop_duplicates(subset=colonne_client).set_index(colonne_client)
    total_client=data_frame[colonne_client]
    total_client=pd.DataFrame(total_client).drop_duplicates(subset=colonne_client).set_index(colonne_client)
    x=total_client[~total_client.index.isin(x.index)]
   
    return x
client_perdu(donne1,'Order Date',0,1,'Customer ID')#date la plus haute dans donne1 est 2018-12-30


In [ ]:
#moyenne delai de traitement des clients perdu plus haute?
client_dans_la_brume=client_perdu(donne1,'Order Date',0,1,'Customer ID')
total_client=donne1.set_index('Customer ID')
client_dans_la_brume=total_client[total_client.index.isin(client_dans_la_brume.index)]#868 commandes que ces clients on passer, il y a un probleme?
client_dans_la_brume

In [ ]:
client_dans_la_brume['Délai entre commande et envoie'].describe()#moyenne plus haute de .1 75% en de 3 jours
donne1['Délai entre commande et envoie'].describe()